In [17]:
import numpy as np
import pandas as pd
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split

from torchvision.datasets import MNIST, FashionMNIST
from matplotlib import pyplot as plt

from nam.wrapper import NAMClassifier, MultiTaskNAMClassifier

In [2]:
random_state = 2016

In [3]:

dataset = pd.read_csv('nam/data/recid.data', delimiter=' ', header=None)
dataset.columns = ["age", "race", "sex", "priors_count", "length_of_stay", "c_charge_degree", "two_year_recid"]

In [4]:
dataset.head()

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,69,6,2,0,1,1,0
1,34,1,2,0,10,1,1
2,24,1,2,4,1,1,1
3,44,6,2,0,1,2,0
4,41,3,2,14,6,1,1


In [5]:
binary = ['sex', 'c_charge_degree']
other = ['age', 'race', 'priors_count', 'length_of_stay']

In [6]:
scaler = MinMaxScaler((-1, 1))
dataset[other] = scaler.fit_transform(dataset[other])
dataset[binary] = dataset[binary] - 1

In [7]:
dataset

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,0.307692,1.0,1,-1.000000,-0.9975,0,0
1,-0.589744,-1.0,1,-1.000000,-0.9750,0,1
2,-0.846154,-1.0,1,-0.789474,-0.9975,0,1
3,-0.333333,1.0,1,-1.000000,-0.9975,1,0
4,-0.410256,-0.2,1,-0.263158,-0.9850,0,1
...,...,...,...,...,...,...,...
6167,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6168,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6169,0.000000,1.0,1,-1.000000,-0.9975,0,0
6170,-0.615385,-1.0,0,-0.842105,-0.9975,1,0


In [8]:
X_data = dataset.drop(columns=['two_year_recid']).to_numpy()
X_data

array([[ 0.30769231,  1.        ,  1.        , -1.        , -0.9975    ,
         0.        ],
       [-0.58974359, -1.        ,  1.        , -1.        , -0.975     ,
         0.        ],
       [-0.84615385, -1.        ,  1.        , -0.78947368, -0.9975    ,
         0.        ],
       ...,
       [ 0.        ,  1.        ,  1.        , -1.        , -0.9975    ,
         0.        ],
       [-0.61538462, -1.        ,  0.        , -0.84210526, -0.9975    ,
         1.        ],
       [-0.87179487,  0.2       ,  0.        , -0.89473684, -0.995     ,
         0.        ]])

In [9]:
y_data = dataset['two_year_recid'].to_numpy().reshape(-1, 1)
y_data

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [10]:
np.zeros((X_data.shape[0],)).shape

(6172,)

In [11]:
def onehot_pos_embedding(x):
    onehot = np.zeros((6, 6))
    for i in range(6):
        onehot[i, i] = 1
    onehot = np.repeat(onehot[np.newaxis, :, :], x.shape[0], axis=0)
    return np.concatenate((x[:, :, np.newaxis], onehot), axis=2)

In [12]:
X_data = onehot_pos_embedding(X_data)
X_data.shape

(6172, 6, 7)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, train_size=0.8, test_size=0.2, random_state=random_state)

In [14]:
X_train.shape

(4937, 6, 7)

In [15]:
onehot = np.zeros((6, 6))
for i in range(6):
    onehot[i, i] = 1
onehot.shape

(6, 6)

In [18]:
dataset = FashionMNIST(root='nam/data/', download=True, train=True)

26422272it [00:05, 4437609.54it/s]                              


Extracting nam/data/FashionMNIST\raw\train-images-idx3-ubyte.gz to nam/data/FashionMNIST\raw



29696it [00:00, 190434.48it/s]                          


Extracting nam/data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to nam/data/FashionMNIST\raw



4422656it [00:01, 2305195.82it/s]                             


Extracting nam/data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to nam/data/FashionMNIST\raw



6144it [00:00, 12277181.41it/s]         

Extracting nam/data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to nam/data/FashionMNIST\raw



In [20]:
dataset.data[0].shape

torch.Size([28, 28])

In [24]:
dataset = MNIST(root='nam/data/', download=True, train=True)
X_data = dataset.data.numpy()
y_data = dataset.targets.numpy().reshape(-1, 1)
# mnist_test = MNIST(root='nam/data/', download=True, train=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, train_size=0.2, test_size=0.8, random_state=random_state, stratify=y_data)

In [35]:
y_train[np.isin(y_train[:, 0], [0, 1])].shape

(2533, 1)

In [6]:
def filterClasses(X, y, classes:list):
    X = X[np.isin(y[:, 0], classes)]
    y = y[np.isin(y[:, 0], classes)]
    return X, y

In [5]:
def cood_encoding(x):
    x = x / 255
    indices = np.zeros((x.shape[1], x.shape[2], 2))
    for i in range(28):
        for j in range(28):
            indices[i, j] = [i, j]
    indices = np.repeat(indices[np.newaxis, :, :, :], x.shape[0], axis=0)
    return np.concatenate((x[:, :, :, np.newaxis], indices), axis=3).reshape(x.shape[0], x.shape[1] * x.shape[2], 3)

In [20]:
X_train, y_train = filterClasses(X_train, y_train, [0, 1])
X_test, y_test = filterClasses(X_test, y_test, [0, 1])

In [21]:

X_train = cood_encoding(X_train)
X_test = cood_encoding(X_test)
X_train.shape, X_test.shape

((2533, 784, 3), (10132, 784, 3))

In [7]:
random_state = 2016
dataset = MNIST(root='nam/data/', download=True, train=True)
X_data = dataset.data.numpy()
y_data = dataset.targets.numpy().reshape(-1, 1)

X_data, y_data = filterClasses(X_data, y_data, [0, 1])

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, train_size=0.2, test_size=0.8, random_state=random_state, stratify=y_data)
X_test, _, y_test, _ = train_test_split(X_test, y_test, train_size=0.1, test_size=0.9, random_state=random_state, stratify=y_test)

X_train = cood_encoding(X_train)
X_test = cood_encoding(X_test)

## Single Task NAMs Classification

In [22]:

model = NAMClassifier(
    num_epochs=1,
    num_learners=2,
    batch_size=256,
    metric='auroc',
    early_stop_mode='max',
    monitor_loss=False,
    n_jobs=10,
    random_state=random_state,
    device='cuda:0'
)
    
model.fit(X_train, y_train)

Format converting and model initializing are done.
(2533, 784, 3) (2533,)


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([256])) must be the same as input size (torch.Size([256, 1]))

In [10]:
pred = model.predict_proba(X_test)
sk_metrics.roc_auc_score(y_test, pred)

## Multitask NAMs Classification

In [11]:
def make_gender_mtl_data(X, y):
    y_male = y.copy()
    y_male[X['sex'] == 1] = np.nan
    y_female = y.copy()
    y_female[X['sex'] == 0] = np.nan
    return pd.concat([y_female, y_male], axis=1)

In [12]:
y_train_mtl = make_gender_mtl_data(X_train, y_train)
y_test_mtl = make_gender_mtl_data(X_test, y_test)

In [13]:
X_train_mtl = X_train.drop(columns=['sex'])
X_test_mtl = X_test.drop(columns=['sex'])

In [14]:
# NaN indicates label missing
y_train_mtl

,two_year_recid,two_year_recid
4819,0.0,NaN
1581,NaN,1.0
0,0.0,NaN
1575,0.0,NaN
1159,1.0,NaN
...,...,...
4604,0.0,NaN
653,0.0,NaN
4691,0.0,NaN
5386,NaN,1.0


In [15]:
model = MultiTaskNAMClassifier(
            num_learners=20,
            patience=60,
            num_epochs=1000,
            num_subnets=10,
            metric='auroc',
            monitor_loss=False,
            early_stop_mode='max',
            n_jobs=10,
            random_state=random_state
        )

model.fit(X_train_mtl, y_train_mtl)

Format converting and model initializing are done.
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])torch.Size([1024, 2, 5, 1])

torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])
torch.Size([1024, 2, 5, 1])


RuntimeError: Tensors must have same number of dimensions: got 3 and 4

In [ ]:
pred = model.predict_proba(X_test_mtl)

In [ ]:
# Flatten and remove nans
y_test_mtl_flat = y_test_mtl.to_numpy().reshape(-1)
pred_flat = pred.reshape(-1)

non_nan_indices = y_test_mtl_flat == y_test_mtl_flat 
y_test_mtl_flat = y_test_mtl_flat[non_nan_indices]
pred_flat = pred_flat[non_nan_indices]

In [ ]:
sk_metrics.roc_auc_score(y_test_mtl_flat, pred_flat)